# Initialisation

In [1]:
from utils.WeaviateManager import VectorManager

/usr/local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
client = VectorManager()

# Create Collection

In [3]:
# Test 1.1 Attempt to create collection with nested schema - error

user_schema = {
    "id_no":"str",
    "age":"int",
    "education":{
        "primary":{
            "school":"str"
        },
        "secondary":"str",
        "tertiary":"str"
    }
}
collection_name = 'Faces'

client.create_collection(collection_name, user_schema)

Invalid date type {'primary': {'school': 'str'}, 'secondary': 'str', 'tertiary': 'str'}


In [4]:
# Test 1.2 Attempt to create collection with invalid data type - error

user_schema = {
    "id_no":"str",
    "age":"int",
    "education": "Unknown Object"
}
collection_name = 'Faces'

client.create_collection(collection_name, user_schema)

Invalid date type Unknown Object


In [5]:
# Test 1.3 Attempt to create collection with data type

user_schema = {
    "id_no":"str",
    "age":"int",
}
collection_name = 'Faces'
client.delete_collection(collection_name)
client.create_collection(collection_name, user_schema)

# Test 1.4 Attempt to create collection with the name collection name - error
user_schema = {
    "id_no":"str",
    "age":"int",
}
collection_name = 'Faces'

client.create_collection(collection_name, user_schema)

Collection does not exist so nothing to delete
Collection Successfully created
Collection has been created


# Create Document

In [6]:
import torch

In [7]:
# Test 2.1 Attempt to create with data that does not abide the schema - error
data_obj = {
    "id_no": "1",
    "age": "1"
}
face_emb = torch.rand(1, 5)
client.create_document(collection_name = 'Faces', properties = data_obj, embedding = face_emb)

Error in creating. Please read error message -> Creating object! Unexpected status code: 422, with response body: {'error': [{'message': "invalid object: invalid integer property 'age' on class 'Faces': requires an integer, the given value is '1'"}]}


In [8]:
# Test 2.2 Attempt to create with original data field
for id_no in range(10):
    face_emb = torch.rand(1, 5)
    if len(face_emb) != 0:
        data_obj = {
            "id_no": "{}".format(id_no),
            "age": id_no
        }
    client.create_document(collection_name = 'Faces', properties = data_obj, embedding = face_emb)

In [9]:
# Test 2.3 Attempt to create with new data field
data_obj = {
    "id_no": "11",
    "new": "2"
}
face_emb = torch.Tensor([0.5766745, 0.9341823, 0.7021697, 0.54776406, 0.013553977])
client.create_document(collection_name = 'Faces', properties = data_obj, embedding = face_emb)

In [10]:
# Test 2.4 Attempt to create data_object with the same name -> error
data_obj = {
    "id_no": "11",
    "new": "2"
}
face_emb = torch.rand(1, 5)
client.create_document(collection_name = 'Faces', properties = data_obj, embedding = face_emb)

This id already existed please use update instead


In [11]:
# Test 2.5 Attempt to create data_object with different embedding length -> Error
data_obj = {
    "id_no": "12",
    "age": 12
}
face_emb = torch.rand(1, 10)
client.create_document(collection_name = 'Faces', properties = data_obj, embedding = face_emb)

Mistmatch vector length, creation failed


# Read Document

In [12]:
# 3.2 Attempt to read a non existent document id number -> error

client.read_document(collection_name, "100")

Attempt to read a non-existent document. No reading is done


In [13]:
# 3.2 Attempt to read a non existent collection -> error

client.read_document('Non-existence', "1")

Attempt to read a non-existent document. No reading is done


In [14]:
# 3.3 Attempt to read one of the file in 2.2

client.read_document(collection_name, "1")

{'class': 'Faces',
 'creationTimeUnix': 1671076617079,
 'id': '1fbf7a0f-1904-4c21-afd8-6bda380e51fd',
 'lastUpdateTimeUnix': 1671076617079,
 'properties': {'age': 1, 'id_no': '1'},
 'vector': [0.12981915, 0.8315664, 0.23063284, 0.3632335, 0.06877518],
 'vectorWeights': None}

In [15]:
# 3.3 Attempt to read one of the file in 2.3

client.read_document(collection_name, "11")

{'class': 'Faces',
 'creationTimeUnix': 1671076617122,
 'id': '9d62d87b-bb17-4736-8714-e1455ffa2b01',
 'lastUpdateTimeUnix': 1671076617122,
 'properties': {'id_no': '11', 'new': '2'},
 'vector': [0.5766745, 0.9341823, 0.7021697, 0.54776406, 0.013553977],
 'vectorWeights': None}

In [16]:
# 3.4 Attempt to read document nearest to the embedding

embedding = torch.Tensor([0.5766745, 0.9341823, 0.7021697, 0.54776406, 0.013553977])

client.get_top_k(collection_name, embedding)

[{'class': 'Faces',
  'creationTimeUnix': 1671076617122,
  'id': '9d62d87b-bb17-4736-8714-e1455ffa2b01',
  'lastUpdateTimeUnix': 1671076617122,
  'properties': {'id_no': '11', 'new': '2'},
  'vector': [0.5766745, 0.9341823, 0.7021697, 0.54776406, 0.013553977],
  'vectorWeights': None,
  'certainty': 0.9999999403953552}]

In [17]:
# 3.5 Attempt to read top k document nearer to the embedding

embedding = torch.Tensor([0.5766745, 0.9341823, 0.7021697, 0.54776406, 0.013553977])

client.get_top_k(collection_name, embedding, 3)

[{'class': 'Faces',
  'creationTimeUnix': 1671076617122,
  'id': '9d62d87b-bb17-4736-8714-e1455ffa2b01',
  'lastUpdateTimeUnix': 1671076617122,
  'properties': {'id_no': '11', 'new': '2'},
  'vector': [0.5766745, 0.9341823, 0.7021697, 0.54776406, 0.013553977],
  'vectorWeights': None,
  'certainty': 0.9999999403953552},
 {'class': 'Faces',
  'creationTimeUnix': 1671076617075,
  'id': '7f4ba177-7ec9-4087-8a96-e90d5a5354e7',
  'lastUpdateTimeUnix': 1671076617075,
  'properties': {'age': 0, 'id_no': '0'},
  'vector': [0.14442688, 0.54120004, 0.42148685, 0.39924592, 0.16699702],
  'vectorWeights': None,
  'certainty': 0.9747301936149597},
 {'class': 'Faces',
  'creationTimeUnix': 1671076617110,
  'id': '963cd308-cf34-4f65-bfcc-a4e31b3d3b6c',
  'lastUpdateTimeUnix': 1671076617110,
  'properties': {'age': 9, 'id_no': '9'},
  'vector': [0.39691275, 0.3834964, 0.54053557, 0.44337738, 0.2580279],
  'vectorWeights': None,
  'certainty': 0.9626578688621521}]

In [18]:
# 3.6 Attempt to read top k document nearer to the wrong embedding -> error

embedding = torch.Tensor([0.5766745, 0.9341823, 0.7021697, 0.54776406])

res = client.get_top_k(collection_name, embedding, 10)

Query with wrong embedding dimension


# Update

In [19]:
# 4.1 Attempt to update non-existence id -> error

update = {
    'id_no': '12',
}

client.update_document(collection_name, update)

Attempt to update a non-existent document. No update is done


In [20]:
# 4.2 Attempt to update with different vector dimension -> error

update = {
    'id_no': '2',
    'vector': torch.rand(1, 3),
}

print(client.read_document(collection_name, "2"))
client.update_document(collection_name, update)
print(client.read_document(collection_name, "2"))

{'class': 'Faces', 'creationTimeUnix': 1671076617083, 'id': 'cd3c8688-7f67-4724-b0e2-34702d5a1258', 'lastUpdateTimeUnix': 1671076617083, 'properties': {'age': 2, 'id_no': '2'}, 'vector': [0.23052144, 0.79224944, 0.57575065, 0.3920939, 0.96631867], 'vectorWeights': None}
Mistmatch vector length, updating failed
{'class': 'Faces', 'creationTimeUnix': 1671076617083, 'id': 'cd3c8688-7f67-4724-b0e2-34702d5a1258', 'lastUpdateTimeUnix': 1671076617279, 'properties': {'age': 2, 'id_no': '2'}, 'vector': [0.23052144, 0.79224944, 0.57575065, 0.3920939, 0.96631867], 'vectorWeights': None}


In [21]:
# 4.3 Attempt to update with new field -> error

update = {
    'id_no': '2',
    'vector': torch.rand(1, 5),
    'non-known': 1
}
print(client.read_document(collection_name, "2"))
client.update_document(collection_name, update)
print(client.read_document(collection_name, "2"))

{'class': 'Faces', 'creationTimeUnix': 1671076617083, 'id': 'cd3c8688-7f67-4724-b0e2-34702d5a1258', 'lastUpdateTimeUnix': 1671076617279, 'properties': {'age': 2, 'id_no': '2'}, 'vector': [0.23052144, 0.79224944, 0.57575065, 0.3920939, 0.96631867], 'vectorWeights': None}
Unknown field(s) in document
{'class': 'Faces', 'creationTimeUnix': 1671076617083, 'id': 'cd3c8688-7f67-4724-b0e2-34702d5a1258', 'lastUpdateTimeUnix': 1671076617279, 'properties': {'age': 2, 'id_no': '2'}, 'vector': [0.23052144, 0.79224944, 0.57575065, 0.3920939, 0.96631867], 'vectorWeights': None}


In [22]:
# 4.4 Attempt to update with only id -> error

update = {
    'id_no': '2',
}

client.update_document(collection_name, update)

Only dict_keys(['id_no']) is found which has nothing to update


In [23]:
# 4.5 Attempt to update age with wrong data type -> error

update = {
    'id_no': '2',
    'age': '2'
}

client.update_document(collection_name, update)

Error in document field(s) -> Update of the object not successful! Unexpected status code: 422, with response body: {'error': [{'message': "msg:bad request code:400 err:invalid integer property 'age' on class 'Faces': requires an integer, the given value is '2'"}]}


In [24]:
# 4.6 Attempt to update age with right data type

update = {
    'id_no': '2',
    'age': 100
}
print(client.read_document(collection_name, "2"))
client.update_document(collection_name, update)
print(client.read_document(collection_name, "2"))

{'class': 'Faces', 'creationTimeUnix': 1671076617083, 'id': 'cd3c8688-7f67-4724-b0e2-34702d5a1258', 'lastUpdateTimeUnix': 1671076617279, 'properties': {'age': 2, 'id_no': '2'}, 'vector': [0.23052144, 0.79224944, 0.57575065, 0.3920939, 0.96631867], 'vectorWeights': None}
{'class': 'Faces', 'creationTimeUnix': 1671076617083, 'id': 'cd3c8688-7f67-4724-b0e2-34702d5a1258', 'lastUpdateTimeUnix': 1671076617348, 'properties': {'age': 100, 'id_no': '2'}, 'vector': [0.23052144, 0.79224944, 0.57575065, 0.3920939, 0.96631867], 'vectorWeights': None}


In [25]:
# 4.6 Attempt to update vector only

update = {
    'id_no': '2',
    'vector': torch.rand(1, 5).numpy(),
}
print(client.read_document(collection_name, "2"))
client.update_document(collection_name, update)
print(client.read_document(collection_name, "2"))

{'class': 'Faces', 'creationTimeUnix': 1671076617083, 'id': 'cd3c8688-7f67-4724-b0e2-34702d5a1258', 'lastUpdateTimeUnix': 1671076617348, 'properties': {'age': 100, 'id_no': '2'}, 'vector': [0.23052144, 0.79224944, 0.57575065, 0.3920939, 0.96631867], 'vectorWeights': None}
{'class': 'Faces', 'creationTimeUnix': 1671076617083, 'id': 'cd3c8688-7f67-4724-b0e2-34702d5a1258', 'lastUpdateTimeUnix': 1671076617373, 'properties': {'age': 100, 'id_no': '2'}, 'vector': [0.43961406, 0.5961837, 0.5770033, 0.82344407, 0.5138211], 'vectorWeights': None}


# Delete

In [26]:
# 5.1 Delete document

client.delete_document(collection_name, "2")
client.read_document(collection_name, "2")

Attempt to read a non-existent document. No reading is done


In [27]:
# 5.2 Delete a nonexistence document -> error

client.delete_document(collection_name, "2")

This id does not exist so no deletion is done


In [28]:
# 5.3 Delete collection

client.delete_collection(collection_name)
client.read_document(collection_name, "1")

Successfully deleted
Attempt to read a non-existent document. No reading is done


In [29]:
# 5.4 Delete non existence collection -> error

client.delete_collection(collection_name)

Collection does not exist so nothing to delete
